In [1]:
using Pkg
Pkg.add("Interpolations")
Pkg.add("BasicInterpolators")

    Updating registry at `C:\Users\Miguel\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Miguel\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Miguel\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Miguel\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Miguel\.julia\environments\v1.9\Manifest.toml`


# Resolvemos para atmósfera estratificada
Siendo $\vec{q}=\nabla(\tau)$ y $\vec{w}$ el vector velocidad del viento, para una atmósfera estratificada se obtiene $\dfrac{dq_x}{dt}=\dfrac{dq_y}{dt}=0;\dfrac{dq_z}{dt}=-q\dfrac{\partial c}{\partial z}-\vec{q}.\dfrac{\partial \vec{w}}{\partial z}$ (Brouwer 2014).

Implementamos un caso de velocidad de viento constante ($\dfrac{\partial \vec{w}}{\partial z}$) con velocidad del sonido dependiente de la altura con la temperatura: $c_{air}=331 \dfrac{m}{s}\sqrt{\dfrac{T_{z=0m}-\dfrac{z}{300m}}{273.15 K}}$


Obtenemos: $\dfrac{dq_z}{dt}=-\sqrt{q_x^2+q_y^2+q_z^2}\dfrac{\partial c}{\partial z}; met=-\dfrac{331}{600*273.15\sqrt{\dfrac{T_{z=0m}-\dfrac{z}{300m}}{273.15 K}}}=- \dfrac{331^2}{600*273.15*c(T,z)}\dfrac{1}{s} \Rightarrow \dfrac{dq_z}{dt}=\dfrac{1}{q_z}\dfrac{dq_z}{dz}=\dfrac{331^2\sqrt{q_{x}^2+q_{y}^2+q_z^2}}{600*273.15*c(T,z)} \Rightarrow \dfrac{dq_z}{dz}=\dfrac{331^2\sqrt{q_{x}^2+q_{y}^2+q_z^2}q_z}{600*273.15*c(T,z)}$

Con $\dfrac{dt}{d\tau}=1$ y la c.i. $\tau(x,y,z=Source)=0$ obtenemos $\tau=| \int \vec{q} d \vec{s} | \approx \Sigma |\vec{q_i}.\vec{\Delta {s_i}}  |  $ siendo s cada elemento diferencial de la trayectoria recorrida por el rayo acústico.

In [1]:
function c_air_TZ(T_z0,z)
    c_air=331*sqrt((T_z0+273.15-z/300)/273.15) #Introduzco T_Z0 en grados centígrados por comodidad
    return c_air
end

c_air_TZ (generic function with 1 method)

Resolvemos la ODE: $\dfrac{dq_x}{dz}=\dfrac{dq_y}{dz}=0 ;\dfrac{dq_z}{dz}=\dfrac{331^2\sqrt{q_{x}^2+q_{y}^2+q_z^2}q_z}{600*273.15*c(T,z)}$

In [2]:
@time begin
#############################################
using DifferentialEquations

#Argumentos y datos
v_wind=[5.,-10,-5.];
T=20.;  
SourcePoint=[0.,0.,0.];
ReceiverPoint=[5000.,5000.,5000.];

#Definimos la ODE 
    function q!(dq, q, p, z)
        c=c_air_TZ(p[1],z)
        dq[1]=0
        dq[2]=0
        dq[3] = 331^2*sqrt(q[1]^2 + q[2]^2 + q[3]^2)*q[3] / (600 * 273.15 *c)
    end
    theta=rad2deg(acos(1/sqrt(3)))
    phi=45 #Calculados directamente
    # Definimos las condiciones iniciales y los parámetros
    q0=1/(c_air_TZ(T, SourcePoint[3])+(v_wind[1]*cosd(phi)+v_wind[2]*sind(phi))*sind(theta))
    q_x = q0*cosd(phi)*sind(theta)
    q_y = q0*sind(phi)*sind(theta)
    q_z = q0*cosd(theta)
    u0 = [q_x, q_y, q_z]
    

    # Definimps el rango de valores de z. Debemos tener cuidado al optimizar, dado
    # que si algún valor (coordenada z) optimizado está fuera de zspan no podemos evaluarlo.
    zspan = (SourcePoint[3], ReceiverPoint[3])

    # Definimos los parámetros
    p = [T] 

    # Definir el problema ODE
    prob = ODEProblem(q!, u0, zspan, p)
    
    #Resolvemos el problema ODE almacenando en los valores de Z que se corresponden a las uniones de los segmentos.
    #z_values=SegmentsCoords[:,3]
    #, saveat=z_values
    sol = solve(prob, RK4(),dt=1e-3)
    #Mirar documentacion para obtener funcion e introducirla como argumento de entrada
    
    #Guardamos los valores de vec(q)
    qx_vals=zeros(length(sol.u))
    qy_vals=zeros(length(sol.u))
    qz_vals=zeros(length(sol.u))

    for i in 1:length(sol.u)
        qx_vals[i]=sol.u[i][1]
        qy_vals[i]=sol.u[i][2]
        qz_vals[i]=sol.u[i][3]
    end
#########################
end

 25.427650 seconds (24.37 M allocations: 1.709 GiB, 4.48% gc time, 13.80% compilation time: 1% of which was recompilation)


In [3]:
using Interpolations
###### qx y qy son constantes
qx=sum(qx_vals)/length(qx_vals)
######
qy=sum(qy_vals)/length(qy_vals)
######
z_values_linear = collect(range(SourcePoint[3], stop=ReceiverPoint[3], length=length(qz_vals)))
qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)




9-element extrapolate(interpolate((::Vector{Float64},), ::Vector{Float64}, Gridded(Linear())), Throw()) with element type Float64:
 0.0016980037155087805
 0.0016980037252445233
 0.001698003822601959
 0.0016980047961770292
 0.001698014531999111
 0.001698111897358504
 0.0016990862654346487
 0.00170890202527781
 0.0017483714615856465

In [4]:
using DifferentialEquations

function Hamiltonian_l(qx::Float64,qy::Float64,qz_z_linear, SegmentsCoords::Array{Float64, 2})
   
    #Calculamos los traveltime entre cada segmento tomando la media de las q entre cada segmento (aproximación regla Trapecio)
    taus=zeros(length(SegmentsCoords[:,1])-1)
    for i in 1:(length(SegmentsCoords[:,1])-1)
        #qx y qy son constantes, da igual que elemento del vector cojamos. Solo debemos tener cuidado con
        aux_x=(SegmentsCoords[i+1,1]-SegmentsCoords[i,1])
        aux_y=(SegmentsCoords[i+1,2]-SegmentsCoords[i,2])
        # La función de interpolación cúbica puede extrapolarse ligeramente fuera del rango de los datos
        #debido a la naturaleza de la interpolación cúbica.  Iterando la condición inicial lleva a BoundsError. Debo emplear 
        #clamp() para asegurarme que interpola dentro del rango
        z1 = clamp(SegmentsCoords[i,3], SegmentsCoords[1,3], SegmentsCoords[end,3])
        z2 = clamp(SegmentsCoords[i+1,3], SegmentsCoords[1,3], SegmentsCoords[end,3])
        aux_z=(SegmentsCoords[i+1,3]-SegmentsCoords[i,3])
        q_m=sqrt(qx^2+qy^2+(qz_z_linear(z1)+qz_z_linear(z2))^2/4)
        taus[i]=q_m*sqrt(aux_x^2+aux_y^2+aux_z^2)
    end
    
    #Sumamos el travel time de cada segmento para obtener el total
    total_travel_time = sum(taus)
        
    return total_travel_time
end

Hamiltonian_l (generic function with 1 method)

In [8]:
@time begin
using Optim

#Argumentos y datos
v_wind=[5.,-10,-5.];
T=20.;  
SourcePoint=[0.,0.,0.];
ReceiverPoint=[5000.,5000.,5000.];

#Optimizamos las coordenadas de las uniones entre segmentos
function objective(SegmentsCoords)
    # Primero nos aseguramos de que SourcePoint y ReceiverPoint no se optimizan pero que se incluyen en SegmentsCoords
    FullSegmentsCoords = vcat(reshape(SourcePoint, 1, :), SegmentsCoords, reshape(ReceiverPoint, 1, :))
    traveltime = Hamiltonian_l(qx, qy, qz_z_linear, FullSegmentsCoords)
    return traveltime
end

#Recta troceada desde la fuente al receptor como condición inicial
num_segments=10;
initial_SegmentsCoords = zeros(num_segments-1, 3); 
for i in 1:num_segments-1
    alpha = i / num_segments
    initial_SegmentsCoords[i, :] = SourcePoint + alpha * (ReceiverPoint - SourcePoint) 
end

# Tenemos que tomar restricciones "de caja" para que los valores de z (SegmentsCoords[:,3]) 
#optimizados no esten fuera del intervalo de solución (zspan)
lower_bounds = fill(SourcePoint[3], size(initial_SegmentsCoords))
upper_bounds = fill(ReceiverPoint[3], size(initial_SegmentsCoords))

# Optimizador interno. Limited-memory Broyden-Fletcher-Goldfarb-Shanno
#método de optimización de segundo orden valido para minimizar funciones no lineales.
inner_optimizer = LBFGS()

# Optimizador con las restricciones "de caja"
optimizer_with_constraints = Fminbox(inner_optimizer)

# Optimizamos con las restricciones
result = optimize(objective, lower_bounds, upper_bounds, initial_SegmentsCoords, optimizer_with_constraints)

#Resultado de la optimización
Optimal_SegmentsCoords = result.minimizer;
    
#Incluimos la fuente y el receptor a las coordenadas optimizadas
Full_Optimal_SegmentsCoords= vcat(reshape(SourcePoint, 1, :), Optimal_SegmentsCoords, reshape(ReceiverPoint, 1, :))
    
#Evaluamos el traveltime para la trayectoria optimizada
traveltime =Hamiltonian_l(qx, qy, qz_z_linear, Full_Optimal_SegmentsCoords)
println("The total travel time for the segmented trajectory is: ", traveltime, " seconds")
end

println("Iteraciones realizadas: ", result.iterations)
println("Evaluaciones de la función: ", result.f_calls)



The total travel time for the segmented trajectory is: 25.493379951972226 seconds
  0.483286 seconds (2.11 M allocations: 211.690 MiB, 18.74% gc time, 25.02% compilation time: 82% of which was recompilation)
Iteraciones realizadas: 2
Evaluaciones de la función: 4031


In [9]:
Full_Optimal_SegmentsCoords

11×3 Matrix{Float64}:
    0.0       0.0       0.0
  504.471   504.471   503.107
 1002.78   1002.78   1000.07
 1502.72   1502.72   1498.65
 2000.95   2000.95   1995.53
 2506.79   2506.79   2500.0
 3133.47   3133.47   3125.0
 3753.18   3753.18   3743.23
 3759.98   3759.98   3750.02
 4384.45   4384.45   4375.0
 5000.0    5000.0    5000.0

In [10]:
@time begin
using Optim
#Argumentos y datos
v_wind=[5.,-10,-5.];
T=20.;  
SourcePoint=[0.,0.,0.];
ReceiverPoint=[5000.,5000.,5000.];

#Optimizamos las coordenadas de las uniones entre segmentos
function objective(SegmentsCoords)
    # Primero nos aseguramos de que SourcePoint y ReceiverPoint no se optimizan pero que se incluyen en SegmentsCoords
    FullSegmentsCoords = vcat(reshape(SourcePoint, 1, :), SegmentsCoords, reshape(ReceiverPoint, 1, :))
    traveltime = Hamiltonian_l(qx, qy, qz_z_linear, FullSegmentsCoords)
    return traveltime
end

#Definimos una matriz de matrices Plot_Coords del tamaño del bucle para luego graficar la distancia de los puntos a una recta
Plot_Coords = Matrix{Matrix{Float64}}(undef, 5, 1)

#Iteramos la condición inicial
initial_SegmentsCoords = zeros(1, 3); #c.i.
for i in 1:5
    if i == 1
        initial_SegmentsCoords[1, :] = SourcePoint + (ReceiverPoint - SourcePoint) / 2
    end
        
    lower_bounds = fill(SourcePoint[3], size(initial_SegmentsCoords))
    upper_bounds = fill(ReceiverPoint[3], size(initial_SegmentsCoords))
    inner_optimizer = LBFGS()
    optimizer_with_constraints = Fminbox(inner_optimizer)
    result = optimize(objective, lower_bounds, upper_bounds, initial_SegmentsCoords, optimizer_with_constraints)
    Optimal_SegmentsCoords = result.minimizer;
    Full_Optimal_SegmentsCoords= vcat(reshape(SourcePoint, 1, :), Optimal_SegmentsCoords, reshape(ReceiverPoint, 1, :))
        
    #Almacenamos las coordenadas de la optimización incluyendo la fuente y el receptor para luego graficar
    Plot_Coords[i]=Full_Optimal_SegmentsCoords
    
    #Evaluamos el travel time para los distintos números de segmentos planteados
    total_time = Hamiltonian_l(qx, qy, qz_z_linear, Full_Optimal_SegmentsCoords)
    println("Num segments: ", Int(size(Full_Optimal_SegmentsCoords, 1) - 1), ". Traveltime:", total_time, " s")
    println("Iteraciones realizadas: ", result.iterations, ". Evaluaciones de la función: ", result.f_calls)
        

    copy_Optimal_SegmentsCoords = zeros(2 * size(Optimal_SegmentsCoords, 1) + 1, 3)
    
    # Calculamos copy_optimal_SegmentsCoords 
    for j in 1:2 * size(Optimal_SegmentsCoords, 1) + 1
        if j==1
            copy_Optimal_SegmentsCoords[j,:]=(SourcePoint+Optimal_SegmentsCoords[1,:])/2
        elseif j==2*size(Optimal_SegmentsCoords,1)+1
            copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[end,:]+ReceiverPoint)/2
        else
            if iseven(j)
                copy_Optimal_SegmentsCoords[j,:]=Optimal_SegmentsCoords[Int(j/2),:]
            else
                copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[Int((j-1)/2),:]+Optimal_SegmentsCoords[Int((j+1)/2),:])/2
            end
        end
    end
    #Aquí iteramos la condición inicial
    initial_SegmentsCoords = copy(copy_Optimal_SegmentsCoords)
end
end




Num segments: 2. Traveltime:25.50428691098754 s
Iteraciones realizadas: 1. Evaluaciones de la función: 72
Num segments: 4. Traveltime:25.49350754090566 s
Iteraciones realizadas: 1. Evaluaciones de la función: 195
Num segments: 8. Traveltime:25.493388986867906 s
Iteraciones realizadas: 2. Evaluaciones de la función: 2704
Num segments: 16. Traveltime:25.49338229836212 s
Iteraciones realizadas: 1. Evaluaciones de la función: 1523
Num segments: 32. Traveltime:25.49337951016519 s
Iteraciones realizadas: 1. Evaluaciones de la función: 1199
  1.687205 seconds (4.48 M allocations: 725.142 MiB, 13.83% gc time, 10.13% compilation time: 100% of which was recompilation)


In [13]:
#Ploteamos la distancia de los puntos a x=y=z(ecuación de la recta para la fuente y el receptor propuesto) para cada Z

#No se puede plantear la formula de la distancia de un punto a una recta dado que si
#por ejemplo se obtiene la coordenada (15,15,10) propondríamos la distancia a la trayectoria recta que pase por el (10,10,10).

#Al plantear la distancia de un punto a una recta para cada Z, es decir, de un punto a la recta x=y, el punto (15,15) tendría
#distancia nula a esa recta. Por tanto, calculamos la distancia como el módulo de la resta de vectores de las coordenadas X,Y. 

using Plots 

plot(legend=:topleft)

z2=Plot_Coords[1][:, 3]
aux2=sqrt.((Plot_Coords[1][:, 1]-Plot_Coords[1][:, 3]).^2+(Plot_Coords[1][:, 2]-Plot_Coords[1][:, 3]).^2)
plot!(z2,aux2, label="2 segments", shape=:diamond)

z4=Plot_Coords[2][:, 3]
aux4=sqrt.((Plot_Coords[2][:, 1]-Plot_Coords[2][:, 3]).^2+(Plot_Coords[2][:, 2]-Plot_Coords[2][:, 3]).^2)
plot!(z4,aux4, label="4 segments", shape=:square)

z8=Plot_Coords[3][:, 3]
aux8=sqrt.((Plot_Coords[3][:, 1]-Plot_Coords[3][:, 3]).^2+(Plot_Coords[3][:, 2]-Plot_Coords[3][:, 3]).^2)
plot!(z8,aux8, label="8 segments", shape=:star6)

z16=Plot_Coords[4][:, 3]
aux16=sqrt.((Plot_Coords[4][:, 1]-Plot_Coords[4][:, 3]).^2+(Plot_Coords[4][:, 2]-Plot_Coords[4][:, 3]).^2)
plot!(z16,aux16, label="16 segments", shape=:octagon)

z32=Plot_Coords[5][:, 3]
aux32=sqrt.((Plot_Coords[5][:, 1]-Plot_Coords[5][:, 3]).^2+(Plot_Coords[5][:, 2]-Plot_Coords[5][:, 3]).^2)
plot!(z32,aux32, label="32 segments", shape=:circle)

xlabel!("z (m)")
ylabel!(" Deviation to a straight line (m)")
#savefig("LISpatialGradientWcte.png")

"C:\\Users\\Miguel\\Desktop\\TFM\\Codigos Julia\\LISpatialGradientWcte.png"

In [15]:
#Ploteamos la distancia de los puntos a x=y=z(ecuación de la recta para la fuente y el receptor propuesto) para cada Z

#No se puede plantear la formula de la distancia de un punto a una recta dado que si
#por ejemplo se obtiene la coordenada (15,15,10) propondríamos la distancia a la trayectoria recta que pase por el (10,10,10).

#Al plantear la distancia de un punto a una recta para cada Z, es decir, de un punto a la recta x=y, el punto (15,15) tendría
#distancia nula a esa recta. Por tanto, calculamos la distancia como el módulo de la resta de vectores de las coordenadas X,Y. 

using Plots 

plot(legend=:topleft)

z2=Plot_Coords[1][:, 3]
aux2=sqrt.((Plot_Coords[1][:, 1]-Plot_Coords[1][:, 3]).^2+(Plot_Coords[1][:, 2]-Plot_Coords[1][:, 3]).^2)
plot!(z2,aux2, label="2 segments", shape=:diamond)

z4=Plot_Coords[2][:, 3]
aux4=sqrt.((Plot_Coords[2][:, 1]-Plot_Coords[2][:, 3]).^2+(Plot_Coords[2][:, 2]-Plot_Coords[2][:, 3]).^2)
plot!(z4,aux4, label="4 segments", shape=:square)

z8=Plot_Coords[3][:, 3]
aux8=sqrt.((Plot_Coords[3][:, 1]-Plot_Coords[3][:, 3]).^2+(Plot_Coords[3][:, 2]-Plot_Coords[3][:, 3]).^2)
plot!(z8,aux8, label="8 segments", shape=:star6)

z16=Plot_Coords[4][:, 3]
aux16=sqrt.((Plot_Coords[4][:, 1]-Plot_Coords[4][:, 3]).^2+(Plot_Coords[4][:, 2]-Plot_Coords[4][:, 3]).^2)
plot!(z16,aux16, label="16 segments", shape=:octagon)

z32=Plot_Coords[5][:, 3]
aux32=sqrt.((Plot_Coords[5][:, 1]-Plot_Coords[5][:, 3]).^2+(Plot_Coords[5][:, 2]-Plot_Coords[5][:, 3]).^2)
plot!(z32,aux32, label="32 segments", shape=:circle)
xlims!(3700,4000)
ylims!(13.90,14.40)
xlabel!("z (m)")
ylabel!(" Deviation to a straight line (m)")
#savefig("LISpatialGradientWcteAmpliada.png")

"C:\\Users\\Miguel\\Desktop\\TFM\\Codigos Julia\\LISpatialGradientWcteAmpliada.png"

In [25]:
Plot_Coords[4]

17×3 Matrix{Float64}:
    0.0       0.0       0.0
  431.175   431.175  1249.99
  862.351   862.351  2499.99
 1175.28   1175.28   2812.49
 1488.2    1488.2    3125.0
 1800.4    1800.4    3400.02
 2112.59   2112.59   3675.03
 2424.78   2424.78   3712.53
 2736.98   2736.98   3750.02
 3049.18   3049.18   4024.48
 3361.37   3361.37   4298.94
 3673.56   3673.56   4336.98
 3985.76   3985.76   4375.02
 4298.4    4298.4    4649.4
 4611.04   4611.04   4923.77
 4805.52   4805.52   4961.89
 5000.0    5000.0    5000.0